<a href="https://colab.research.google.com/github/Boyintheorchard/Image_Translator/blob/main/AI_MetaPrep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**📦 Objective**

Build a repeatable, AI-powered workflow to:

Process multi-angle bag images

Extract and merge metadata from Excel (dimensions, color, style)

Generate structured AI descriptions using GPT-4o

Output a CSV ready for:

Prompt libraries

AI image generation (e.g., RunwayML, Veo, Midjourney)

Internal QA or cataloging



---



**🛠 Core Tools Used**

Google Colab (for development)

OpenAI GPT-4o API (for image-to-text)

Pandas (data handling)

Base64 / PIL (image encoding)

Excel metadata sheets with:

Article Level: Height/Width/Depth

SKU Level: Color

SKU Classification: Color Code



---



**🔁 Workflow Steps**

📂 Step 1: Prepare & Upload Files
1. Open your local copy of Item Master
2. Under **SKU Classification** Sheet/Tab;
  1. Change **AQ1** to Colour
  2. Change **AR1** to COL
  3. Change **AS1** to CODE
3. Upload New Item Master .xlsx file (with all metadata sheets)
4. Upload ZIP file of bag images

📊 Step 2: Extract Metadata `[Code]`

Parse image filenames: Article-Insertion-ColourCode-View

Merge across 3 sheets: Article, SKU Level, SKU Classification

Add Dimensions column (in cm)

Clean for duplicates and inconsistent color code matches

🤖 Step 3: AI Description via GPT-4o `[Code]`

Loop through each image

Use dimensions + color + view as input context

Get back 1-sentence professional descriptions

Add column: AI_Description

🧾 Step 4: Output
Export full CSV as:
Preprocessed_Bag_Metadata.csv or Final_Metadata_With_AI_Descriptions.csv

Each row includes:

Image file

Article No, Color, Colour Code, View

Dimensions

AI Description (GPT-4o)



In [ ]:
# 📦 Colab Preprocessing Script: Bag Image Metadata Join (Safe Merge Version)
import pandas as pd, os, re, zipfile
from google.colab import files

# ==== STEP 1: Upload Files ====
print("🗂 Upload your METADATA Excel (.xlsx) file:")
meta_uploaded = files.upload()
meta_path = list(meta_uploaded.keys())[0]

print("🖼 Upload your ZIP of images:")
zip_uploaded = files.upload()
zip_path = list(zip_uploaded.keys())[0]

# ==== STEP 2: Unzip Images ====
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall("images")

image_folder = "images"
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# ==== STEP 3: Parse Filenames ====
pattern = re.compile(r"^(?P<Article>[A-Z]{2}\d-\d+)(?:-(?P<Insertion>\d+))?-(?P<ColourCode>\d+)-(?P<ViewNo>\d+)", re.I)

parsed = []
for fname in image_files:
    m = pattern.match(fname)
    parsed.append({
        "Image File": fname,
        "Article No": m.group("Article") if m else "",
        "Insertion": m.group("Insertion") if m and m.group("Insertion") else "",
        "Colour Code": m.group("ColourCode") if m else "",
        "View No": m.group("ViewNo") if m else ""
    })

images_df = pd.DataFrame(parsed)

# ==== STEP 4: Load Excel Sheets ====
xls = pd.ExcelFile(meta_path)

# Article Dimensions
article_df = pd.read_excel(xls, sheet_name="Artlcle Level (R&D)", header=1)
article_df = article_df.rename(columns={"Article No.": "Article No"})

article_dim = article_df[["Article No", "Height (MM)", "Width (MM)", "Depth (MM)"]].dropna(subset=["Article No"])
article_dim = article_dim.rename(columns={
    "Height (MM)": "Height (cm)",
    "Width (MM)": "Width (cm)",
    "Depth (MM)": "Depth (cm)"
})
for col in ["Height (cm)", "Width (cm)", "Depth (cm)"]:
    article_dim[col] = pd.to_numeric(article_dim[col], errors="coerce") / 10

# SKU Level: Color info
sku_df = pd.read_excel(xls, sheet_name="SKU Level (R&D)", header=1)
sku_df = sku_df.rename(columns={"Article No.": "Article No"})
sku_df["Color"] = sku_df["Color"].str.strip()

# SKU Classification: Color Code info
code_df = pd.read_excel(xls, sheet_name="SKU Classification")
code_df.columns = code_df.columns.str.strip()
code_df = code_df.rename(columns={"Colour": "Color", "CODE": "Color Code"})
code_df["Color"] = code_df["Color"].str.strip()

# ==== STEP 5: Merge Color + Color Code safely ====
# Step 1: Merge color with color code
# Ensure the index used for mapping is unique
code_df_unique_color = code_df.drop_duplicates(subset=["Color"])
sku_df["Colour Code"] = sku_df["Color"].map(code_df_unique_color.set_index("Color")["Color Code"])


# Step 2: Drop duplicates to avoid many-to-many explosion
sku_unique = sku_df.drop_duplicates(subset=["Article No", "Colour Code"])

# ==== STEP 6: Merge with Image Files ====
merged = (
    images_df
    .merge(sku_unique[["Article No", "Color", "Colour Code"]], on=["Article No", "Colour Code"], how="left")
    .merge(article_dim, on="Article No", how="left")
)

# Optional: Map View No to readable label
view_map = {"1": "Front View", "2": "Side View", "3": "Back View"}
merged["View Description"] = merged["View No"].map(view_map)

# Empty column for next stage
merged["AI Prompt"] = ""

from openai import OpenAI
import base64
import os

# Replace with your own key
client = OpenAI(api_key="INSERT CHAT GPT API")

def encode_image(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode("utf-8")

merged["Dimensions"] = merged.apply(
    lambda row: f"{round(row['Height (cm)']/10, 1)}cm (H) x {round(row['Width (cm)']/10, 1)}cm (W) x {round(row['Depth (cm)']/10, 1)}cm (D)",
    axis=1
)

def gpt_caption(image_path, prompt="Describe this fashion bag’s color, shape, material, and design details. Mention hardware or structure if visible."):
    base64_image = encode_image(image_path)
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": [
                {"type": "text", "text": prompt},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
            ]}
        ],
        max_tokens=200
    )
    return response.choices[0].message.content

test_df = merged.head(16).copy()

test_df["AI_Description"] = test_df.apply(
    lambda row: gpt_caption(
        f"images/{row['Image File']}",
        prompt=(
            f"Describe this fashion bag using industry terms. Include:\n"
            f"- Color: {row['Color']}\n"
            f"- Shape/silhouette\n"
            f"- Material (if visible)\n"
            f"- Handle or strap type\n"
            f"- Any visible details (e.g. quilting, buckles, metalwork)\n"
            f"- Use these measurements: {row['Dimensions']}.\n"
            f"Return one coherent sentence."
        )
    ),
    axis=1
)

merged["Dimensions"] = merged.apply(
    lambda row: f"{int(row['Height (cm)'])}cm (H) x {int(row['Width (cm)'])}cm (W) x {int(row['Depth (cm)'])}cm (D)",
    axis=1
)

# Caption each image using GPT-4 Vision
test_df["AI_Description"] = test_df["Image File"].apply(lambda fname: gpt_caption(f"images/{fname}"))

# Export results
test_df.to_csv("Preprocessed_with_AI_Descriptions.csv", index=False)
from google.colab import files
files.download("Preprocessed_with_AI_Descriptions.csv")

merged["AI_Description"] = merged.apply(
    lambda row: gpt_caption(
        f"images/{row['Image File']}",
        prompt=(
            f"Describe this fashion bag using industry terms. Include:\n"
            f"- Color: {row['Color']}\n"
            f"- Shape/silhouette\n"
            f"- Material (if visible)\n"
            f"- Handle or strap type\n"
            f"- Any visible details (e.g. quilting, buckles, metalwork)\n"
            f"- Use these measurements: {row['Dimensions']}.\n"
            f"Return one coherent sentence."
        )
    ),
    axis=1
)

# ==== STEP 7: Export Final CSV ====
output_name = "Preprocessed_Bag_Metadata.csv"
merged.to_csv(output_name, index=False)
files.download(output_name)

print("✅ Done! Clean CSV generated and downloading.")